# 00. PDF抽出（Extract from PDF）

---

## 📌 TL;DR（このNotebookでやること）

日本看護協会「病院看護実態調査」PDFから、分析に必要な **都道府県別テーブル** を抽出し、後続処理で再利用できる **raw CSV** として保存する。

---

## 🔎 Notebook Purpose

本プロジェクトの上流（Raw層）として、PDFという非構造データを **機械可読な表データ** に変換する。
以降の `02_data_validation.ipynb` / `01_create_master_dataset.ipynb` は、本Notebookの出力に依存する。

---

## 📥 Input（前提）

- PDF（配置場所）：`data/raw/日本看護協会_病院看護実態調査_離職率等_2025.pdf`
- PDF内の表は **ページ番号（0-index）** で指定して抽出する  
  ※ PDFの版が変わるとページがズレる可能性があるため、必要に応じて「Page Settings」を更新する

---

## 📤 Output（data/raw）

- `日本看護協会_離職率_都道府県別_2023.csv`
- `日本看護協会_夜勤72h超過率_都道府県別_2024.csv`  
  - 表50（夜勤72h超過率）＋ 表44/47（三交代/二交代の月平均夜勤回数）を **同一ファイルに統合**
- `日本看護協会_給与_都道府県別_2024.csv`  
  - 表34/37/40（平均税込給与総額）を統合

---

## 🧩 Extraction Design（処理設計）

1. **Path Settings**：プロジェクトルート推定 → `data/raw` を解決  
2. **Page Settings**：抽出対象のページ（0-index）を定義  
3. **Extract**：離職率 / 夜勤関連 / 給与 の表を抽出  
4. **Save**：後続ノートで扱いやすい raw CSV として保存  
5. **(Optional) Debug**：抽出失敗時の診断補助

---

## ⚠ Notes（重要）

- 本Notebookは「PDF抽出」なので、**同じ表でもPDFの構造（罫線/列結合/注記）** により抽出結果が変動し得る  
- ここでのCSVは **“統合前のRaw成果物”** とし、整形・定義の確定は `02_data_validation.ipynb` 側で行う

---


In [14]:
# ===============================
# Imports
# ===============================
from pathlib import Path
import pandas as pd
import pdfplumber


In [15]:
# ===============================
# Path Settings
# ===============================

# 実行環境に依存しない「冪等性」と「ポータビリティ」を確保するため、パスを動的に解決
cwd = Path.cwd().resolve()

# カレントディレクトリが notebooks 層の場合、1階層上を「プロジェクトルート（BASE_DIR）」に設定
if cwd.name == "notebooks":
    BASE_DIR = cwd.parent
# 直下に主要ディレクトリが揃っている場合、現在の場所をプロジェクトルートと判定
elif (cwd / "notebooks").exists() and (cwd / "data").exists():
    BASE_DIR = cwd
else:
    # 適切なディレクトリ階層を見つけるまで上位へ遡る「再帰的な探索」によるルート特定
    BASE_DIR = None
    for p in [cwd] + list(cwd.parents):
        if (p / "data").exists() and (p / "notebooks").exists():
            BASE_DIR = p
            break
    # 探索失敗時のフォールバック（例外処理の代わりの安全策）として現在地を指定
    if BASE_DIR is None:
        BASE_DIR = cwd  

# メダリオンアーキテクチャに基づき、ソースデータ（Raw）と加工後データ（Processed）のパスを定義
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"

# ディレクトリの存在を保証し、パイプライン実行時のエラーを未然に防ぐ（例外防止処理）
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# データソース（Rawデータ）のパスを定義。実務では不変（Immutable）なファイルとして扱う
pdf_path = RAW_DIR / "日本看護協会_病院看護実態調査_離職率等_2025.pdf"

# ファイルの存在有無をチェックする「バリデーション」を実施し、異常があれば即座に処理を中断（Fail Fast）
if not pdf_path.exists():
    raise FileNotFoundError(f"PDF not found: {pdf_path}")

# デバッグおよびログ出力：本番運用時のトラブルシューティングに必要な情報を標準出力
print("✅ Path resolved")
print("BASE_DIR:", BASE_DIR)
print("RAW_DIR :", RAW_DIR)
print("PDF     :", pdf_path)

✅ Path resolved
BASE_DIR: /Users/hideomi.h/nurse-turnover-analysis
RAW_DIR : /Users/hideomi.h/nurse-turnover-analysis/data/raw
PDF     : /Users/hideomi.h/nurse-turnover-analysis/data/raw/日本看護協会_病院看護実態調査_離職率等_2025.pdf


In [16]:
# ===============================
# Page Settings（0-index = PDF表示ページ-1）
# ===============================
P_TURNOVER = 73    # PDF 74ページ（表8）離職率
P_NIGHT72P = 108   # PDF 109ページ（表50）夜勤72h超過率

P_SAL_T34  = 96    # PDF 97ページ（表34）新卒初任給（高卒＋3年課程）
P_SAL_T37  = 98    # PDF 99ページ（表37）新卒初任給（大卒）
P_SAL_T40  = 100   # PDF 101ページ（表40）勤続10年・非管理職 月額給与

P_AVG_T44  = 103   # PDF 104ページ（表44）三交代制 月平均夜勤回数
P_AVG_T47  = 105   # PDF 106ページ（表47）二交代制 月平均夜勤回数


In [17]:
# ===============================
# Helper Functions
# ===============================

# データの汚染を防ぐため、集計行や欠損値などの「分析対象外レコード」を定義（定数管理）
EXCLUDE_PREF = {"計", "未回答", "無回答・不明", ""}

def extract_table(page_idx: int, table_idx: int = 0) -> pd.DataFrame:
    """指定ページの指定テーブルを抽出し、DataFrame化（全セル文字列＋strip）"""
    # コンテキストマネージャを使用し、PDFリソースの解放を確実に実行
    with pdfplumber.open(pdf_path) as pdf:
        tables = pdf.pages[page_idx].extract_tables()

    # 抽出結果が空の場合、後続処理でのエラーを防ぐため即座に例外をスロー（ガード節）
    if not tables:
        raise ValueError(f"No tables found on page_idx={page_idx}")

    # インデックス外参照を防ぐ境界値チェックを実施
    if table_idx >= len(tables):
        raise ValueError(f"table_idx={table_idx} not found on page_idx={page_idx}. tables_found={len(tables)}")

    # 2次元リストをDataFrameへ変換し、欠損値（None）を空文字に正規化
    t = tables[table_idx]
    df = pd.DataFrame(t).replace({None: ""})
    # 全セルの余計な空白を削除する「データトリミング」を一括適用
    df = df.map(lambda x: str(x).strip())
    return df


def filter_pref(df: pd.DataFrame, col: str = "prefecture") -> pd.DataFrame:
    # フィルタリングの精度を上げるため、対象列の型キャストとトリミングを再実施
    df[col] = df[col].astype(str).str.strip()
    # 定数 EXCLUDE_PREF に基づき、分析に不要なノイズデータを除外（ノイズフィルタリング）
    return df[~df[col].isin(EXCLUDE_PREF)].copy()


def pct_to_float(x: str):
    """'11.3%' -> 11.3 / '' -> NA"""
    # 欠損値に対する安全なハンドリング（Null Safety）
    if x is None:
        return pd.NA
    # 全角「％」など表記揺れを吸収する「正規化」処理
    s = str(x).strip().replace("％", "%")
    if s == "":
        return pd.NA
    s = s.replace("%", "")
    try:
        # 文字列型から浮動小数点型へのパース（型変換）
        return float(s)
    except:
        # パース失敗時に異常終了させず、欠損値として扱うことでパイプラインの停止を回避
        return pd.NA


def yen_to_float(x: str):
    """'276,127' -> 276127 / '' -> NA"""
    if x is None:
        return pd.NA
    # 桁区切りのカンマなど、計算の邪魔になる記号を除去（サニタイズ）
    s = str(x).replace(",", "").strip()
    if s == "":
        return pd.NA
    try:
        return float(s)
    except:
        return pd.NA


def num_to_float(x: str):
    """'7.4' -> 7.4 / '' -> NA"""
    if x is None:
        return pd.NA
    s = str(x).replace(",", "").strip()
    if s == "":
        return pd.NA
    try:
        return float(s)
    except:
        return pd.NA


def extract_pref_value_by_col(
    page_idx: int,
    table_idx: int,
    row_start: int,
    pref_col: int,
    value_col: int,
    value_name: str,
    converter=None,
) -> pd.DataFrame:
    """都道府県列 + 值列を、列番号指定で安定抽出"""
    # モジュール化された抽出関数を呼び出し、再利用性を確保
    df = extract_table(page_idx, table_idx=table_idx)

    # スキーマ（表の形）が想定と異なる場合に早期検知する「スキーマチェック」
    if df.shape[0] <= row_start or df.shape[1] <= max(pref_col, value_col):
        raise ValueError(f"Table shape looks wrong on page_idx={page_idx}: shape={df.shape}")

    # 不要なヘッダー行をスキップし、必要な列だけを「スライシング」して抽出
    data = df.iloc[row_start:].copy()
    out = pd.DataFrame({
        "prefecture": data[pref_col],
        value_name: data[value_col],
    })

    # 前述のフィルタリング関数を適用し、ビジネスロジックに合致するレコードのみを抽出
    out = filter_pref(out, "prefecture")

    # 指定されたコンバータ（関数）を適用し、列単位のデータ変換（ベクトル演算）を実行
    if converter is not None:
        out[value_name] = out[value_name].map(converter)

    # データの「欠損率」を算出し、抽出プロセスが正常かを可視化（データ品質モニタリング）
    na_rate = float(out[value_name].isna().mean())
    print(f"{value_name}: rows={len(out)}  NA_rate={na_rate:.3f}")
    return out

In [18]:
# ===============================
# Extract: Turnover（表8）
# ===============================
print("Extracting: turnover (table 8) ...")

df_t = extract_table(P_TURNOVER, table_idx=0)

# 既存の安定パターン：ヘッダ2行を落とし、# ===============================
# Extract: Turnover（表8）
# ===============================
# パイプラインの実行状況を可視化するための「進捗ログ」を出力
print("Extracting: turnover (table 8) ...")

# 外部定義された定数 P_TURNOVER（ページ番号）を使用し、PDFから「生データ」を抽出
df_t = extract_table(P_TURNOVER, table_idx=0)

# スキーマ構造に基づき、非データ行（2行のヘッダー）を「スライス」してデータ部のみを特定
data_t = df_t.iloc[2:].copy()

# 非構造化データから、必要な属性（列）のみを選択して構造化（プロジェクション）
turnover = pd.DataFrame({
    "prefecture": data_t[0],             # 都道府県名：主キー（JOINのキー）となる列
    "turnover_total": data_t[2],        # 常勤看護師の離職率
    "turnover_new_grad": data_t[4],     # 新卒者の離職率
    "turnover_experienced": data_t[6],  # 既卒者（中途）の離職率
})

# 事前に定義したフィルタリング関数で、集計行などの「分析対象外ノイズ」を除去
turnover = filter_pref(turnover, "prefecture")

# 複数の列に対して一括で型変換（文字列 → 浮動小数点）を適用し、数値計算が可能な状態へ整形
for c in ["turnover_total", "turnover_new_grad", "turnover_experienced"]:
    turnover[c] = turnover[c].map(pct_to_float)

# 処理後のデータ量（レコード数・カラム数）を確認し、データの欠落がないかチェック（整合性確認）
print("turnover shape:", turnover.shape)

# 中間結果をプレビューし、期待通りのデータ変換が行われているかを視覚的に確認
display(turnover.head(3))
data_t = df_t.iloc[2:].copy()

turnover = pd.DataFrame({
    "prefecture": data_t[0],
    "turnover_total": data_t[2],
    "turnover_new_grad": data_t[4],
    "turnover_experienced": data_t[6],
})

turnover = filter_pref(turnover, "prefecture")
for c in ["turnover_total", "turnover_new_grad", "turnover_experienced"]:
    turnover[c] = turnover[c].map(pct_to_float)

print("turnover shape:", turnover.shape)
display(turnover.head(3))


Extracting: turnover (table 8) ...
Extracting: turnover (table 8) ...
turnover shape: (47, 4)


,prefecture,turnover_total,turnover_new_grad,turnover_experienced
3,北海道,11.5,5.9,16.6
4,青森県,8.6,10.7,16.7
5,岩手県,6.8,7.8,19.1


turnover shape: (47, 4)


,prefecture,turnover_total,turnover_new_grad,turnover_experienced
3,北海道,11.5,5.9,16.6
4,青森県,8.6,10.7,16.7
5,岩手県,6.8,7.8,19.1


In [19]:
# ===============================
# Extract: Night（表50 / 表44 / 表47）
#  - 表50: 夜勤72h超過率（%）
#  - 表44: 三交代制 月平均夜勤回数（回/月）
#  - 表47: 二交代制 月平均夜勤回数（回/月）
# ===============================

print("Extracting: night_72h_plus (table 50) ...")
df_n = extract_table(P_NIGHT72P, table_idx=0)
data_n = df_n.iloc[1:].copy()  # 表50はこの開始が安定（既存実装）
night_72h = pd.DataFrame({
    "prefecture": data_n[0],
    "night_shift_72h_plus": data_n[5],
})
night_72h = filter_pref(night_72h, "prefecture")
night_72h["night_shift_72h_plus"] = night_72h["night_shift_72h_plus"].map(pct_to_float)
print("night_72h shape:", night_72h.shape)


print("Extracting: night shift counts (table 44/47) ...")
# ユーザー提示の確定仕様（表示結果から確定）
night_3shift = extract_pref_value_by_col(
    page_idx=P_AVG_T44, table_idx=0, row_start=2, pref_col=0, value_col=2,
    value_name="night_shifts_per_month_three_shift",
    converter=num_to_float,
)

night_2shift = extract_pref_value_by_col(
    page_idx=P_AVG_T47, table_idx=0, row_start=2, pref_col=0, value_col=2,
    value_name="night_shifts_per_month_two_shift",
    converter=num_to_float,
)

# 同一ファイルに統合（要件）
night_master = (
    night_72h
    .merge(night_3shift, on="prefecture", how="left")
    .merge(night_2shift, on="prefecture", how="left")
)

print("night_master shape:", night_master.shape)
display(night_master.head(3))


Extracting: night_72h_plus (table 50) ...
night_72h shape: (47, 2)
Extracting: night shift counts (table 44/47) ...
night_shifts_per_month_three_shift: rows=47  NA_rate=0.000
night_shifts_per_month_two_shift: rows=47  NA_rate=0.000
night_master shape: (47, 4)


,prefecture,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
0,北海道,36.7,7.8,4.6
1,青森県,36.5,7.7,4.8
2,岩手県,11.8,7.5,4.1


In [20]:
# ===============================
# Extract: Salary（表34 / 表37 / 表40）
#  - いずれも「平均税込給与総額（円）」 = 列3
# ===============================

print("Extracting: salary tables (t34/t37/t40) ...")

salary_t34 = extract_pref_value_by_col(
    page_idx=P_SAL_T34, table_idx=0, row_start=2, pref_col=0, value_col=3,
    value_name="starting_salary_nurse_diploma_monthly_yen",
    converter=yen_to_float,
)

salary_t37 = extract_pref_value_by_col(
    page_idx=P_SAL_T37, table_idx=0, row_start=2, pref_col=0, value_col=3,
    value_name="starting_salary_nurse_bachelor_monthly_yen",
    converter=yen_to_float,
)

salary_t40 = extract_pref_value_by_col(
    page_idx=P_SAL_T40, table_idx=0, row_start=2, pref_col=0, value_col=3,
    value_name="salary_nurse_10yr_non_manager_monthly_yen",
    converter=yen_to_float,
)

salary_master = (
    salary_t34
    .merge(salary_t37, on="prefecture", how="left")
    .merge(salary_t40, on="prefecture", how="left")
)

print("salary_master shape:", salary_master.shape)
display(salary_master.head(3))


Extracting: salary tables (t34/t37/t40) ...
starting_salary_nurse_diploma_monthly_yen: rows=47  NA_rate=0.000
starting_salary_nurse_bachelor_monthly_yen: rows=47  NA_rate=0.000
salary_nurse_10yr_non_manager_monthly_yen: rows=47  NA_rate=0.000
salary_master shape: (47, 4)


,prefecture,starting_salary_nurse_diploma_monthly_yen,starting_salary_nurse_bachelor_monthly_yen,salary_nurse_10yr_non_manager_monthly_yen
0,北海道,270231.0,276652.0,326530.0
1,青森県,270494.0,275476.0,314135.0
2,岩手県,268264.0,276926.0,322466.0


In [8]:
# ===============================
# Save to CSV（data/raw）
# ===============================

# 期待する列が揃っているか（要件チェック）
required_cols = {
    "night_shift_72h_plus",
    "night_shifts_per_month_three_shift",
    "night_shifts_per_month_two_shift",
}
missing_cols = [c for c in required_cols if c not in night_master.columns]
if missing_cols:
    raise RuntimeError(f"night_master is missing columns: {missing_cols}")

out_turnover = RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv"
out_night    = RAW_DIR / "日本看護協会_夜勤72h超過率_都道府県別_2024.csv"
out_salary   = RAW_DIR / "日本看護協会_給与_都道府県別_2024.csv"

turnover.to_csv(out_turnover, index=False, encoding="utf-8-sig")
night_master.to_csv(out_night, index=False, encoding="utf-8-sig")
salary_master.to_csv(out_salary, index=False, encoding="utf-8-sig")

print("✅ Saved (raw)")
print(f"- {out_turnover}  shape={turnover.shape}")
print(f"- {out_night}     shape={night_master.shape}")
print(f"- {out_salary}    shape={salary_master.shape}")

# 書き込み先を取り違えていないか最終確認（読み戻し）
night_check = pd.read_csv(out_night)
print("✅ Reload check (night file) columns:", list(night_check.columns))
display(night_check.head(3))


✅ Saved (raw)
- /Users/hideomi.h/nurse-turnover-analysis/data/raw/日本看護協会_離職率_都道府県別_2023.csv  shape=(47, 4)
- /Users/hideomi.h/nurse-turnover-analysis/data/raw/日本看護協会_夜勤72h超過率_都道府県別_2024.csv     shape=(47, 4)
- /Users/hideomi.h/nurse-turnover-analysis/data/raw/日本看護協会_給与_都道府県別_2024.csv    shape=(47, 4)
✅ Reload check (night file) columns: ['prefecture', 'night_shift_72h_plus', 'night_shifts_per_month_three_shift', 'night_shifts_per_month_two_shift']


,prefecture,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
0,北海道,36.7,7.8,4.6
1,青森県,36.5,7.7,4.8
2,岩手県,11.8,7.5,4.1


In [21]:
# ===============================
# (Optional) Debug Helper
# ===============================
def show_tables_on_page(page_idx: int):
    with pdfplumber.open(pdf_path) as pdf:
        tables = pdf.pages[page_idx].extract_tables()
    print(f"page_idx={page_idx}  tables_found={len(tables)}")
    for i, t in enumerate(tables):
        df = pd.DataFrame(t).replace({None: ""}).map(lambda x: str(x).strip())
        print(f"\n--- table_idx={i}  shape={df.shape} ---")
        display(df.head(8))


## ✅ Project Status（現時点）

このNotebookは、PDF由来データを **再現可能な形でraw CSV化** する上流工程である。
後続の品質検査・統合処理は以下で実施する。

- `02_data_validation.ipynb`：抽出結果の検証（欠損・列構造・単位・除外ルール）
- `01_create_master_dataset.ipynb`：複数ソースを結合し master dataset を生成

---

## ⚠ Limitations

- PDF版やレイアウト変更により、ページ番号・抽出精度が変わる可能性がある
- 抽出は「表構造」に依存するため、注記や列結合の揺れを完全には吸収できない

---

## 🚀 Next（改善案）

- 抽出後のスキーマを固定し、`02`側で **スキーマ検証（期待列・型・値域）** を自動テスト化
- PDF差分（版更新）に備え、ページ設定の検知/更新を半自動化（例：表タイトル検索）
